<a href="https://colab.research.google.com/github/jsebastianquiroga/PUJ_NLP_IA/blob/main/proyecto/inferencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import gdown
import joblib

# Check for GPU and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

def generate_embeddings(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().reshape(-1)

# Download and load the data and models
gdown.download('https://drive.google.com/uc?id=1tsSItM8O_VeDI3mjJ87gFObYFe7FdJ7q', 'knn_model.pkl', quiet=False)
knn_model = joblib.load('knn_model.pkl')

gdown.download('https://drive.google.com/uc?id=1E-KjJCjGno41DchF_lr55FKU7JkY-IEf', 'merged_df_embeddings.parquet', quiet=False)
merged_df = pd.read_parquet('merged_df_embeddings.parquet')

# Combine 'description' and 'categories' text, then generate embeddings
merged_df['combined_text'] = merged_df['description'] + " " + merged_df['categories']
merged_df['combined_embedding'] = merged_df['combined_text'].apply(generate_embeddings)

# Train the k-NN model
knn_model = NearestNeighbors(n_neighbors=6)  # Fetching 6 neighbors including the book itself
knn_model.fit(np.stack(merged_df['combined_embedding'].values))

def recommend_books(new_text):
    new_embedding = generate_embeddings(new_text)
    distances, indices = knn_model.kneighbors([new_embedding])
    recommended_books = merged_df.iloc[indices[0][1:]]  # Exclude the first index as it will be the book itself
    return recommended_books[['Title', 'authors', 'publishedDate']]

# Example usage
recommend_books('I want to read something about fantasy and love')

Using device: cuda


Downloading...
From: https://drive.google.com/uc?id=1tsSItM8O_VeDI3mjJ87gFObYFe7FdJ7q
To: /content/knn_model.pkl
100%|██████████| 417M/417M [00:10<00:00, 40.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1E-KjJCjGno41DchF_lr55FKU7JkY-IEf
To: /content/merged_df_embeddings.parquet
100%|██████████| 546M/546M [00:09<00:00, 58.3MB/s]


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import gdown
import joblib

# Ensure GPU is available
if not torch.cuda.is_available():
    raise ValueError("No GPU found, please ensure you're running on a GPU instance.")

device = torch.device("cuda")

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

def generate_embeddings(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    tokens = {key: val.to(device) for key, val in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().reshape(-1)

# Download and load the data and models
gdown.download('https://drive.google.com/uc?id=1tsSItM8O_VeDI3mjJ87gFObYFe7FdJ7q', 'knn_model.pkl', quiet=False)
knn_model = joblib.load('knn_model.pkl')

gdown.download('https://drive.google.com/uc?id=1E-KjJCjGno41DchF_lr55FKU7JkY-IEf', 'merged_df_embeddings.parquet', quiet=False)
merged_df = pd.read_parquet('merged_df_embeddings.parquet')

# Define the recommendation function
def recommend_books_1(new_text):
    new_embedding = generate_embeddings(new_text)
    distances, indices = knn_model.kneighbors([new_embedding])

    # Normalize the review scores to [0, 1]
    max_score = merged_df['review/score'].max()
    min_score = merged_df['review/score'].min()
    merged_df['normalized_score'] = (merged_df['review/score'] - min_score) / (max_score - min_score)

    # Adjust distances based on review scores
    for i, index in enumerate(indices[0]):
        distances[0][i] *= (1 - merged_df.iloc[index]['normalized_score'])

    # Now, sort the books based on adjusted distances
    sorted_indices = np.argsort(distances[0])

    # Get top 5 books based on adjusted distances
    recommended_books = merged_df.iloc[sorted_indices[1:6]]  # Exclude the first index as it will be the book itself
    return recommended_books[['Title', 'authors', 'publishedDate']]

# Example usage
print(recommend_books_1('I want to read something fantasy and love'))
